In [75]:
with open("17.txt") as i:
    q_input = i.read().splitlines()

In [76]:
from dataclasses import dataclass
from copy import deepcopy
import itertools

@dataclass
class Coord:
    x: int
    y: int
    def __add__(self, other):
        return Coord(self.x + other.x, self.y + other.y)
    def __sub__(self, other):
        return Coord(self.x - other.x, self.y - other.y)
    def __hash__(self):
        return hash((self.x, self.y))
    
class Rock:

    def __init__(self, name, coords):
        self.name = name
        self.coords = [Coord(x,y) for x,y in coords]
        self.old_coords = []

    @property
    def x_values(self):
        return [coord.x for coord in self.coords]

    @property
    def y_values(self):
        return [coord.y for coord in self.coords]
    
    def transform(self,vector):
        self.old_coords = self.coords
        self.coords = [coord+vector for coord in self.coords]
    
    def revert(self):
        self.coords = self.old_coords
    
    def __repr__(self):
        return f"Rock({self.name})"

def rock_list():
    ROCKS = [
        Rock(name="-", coords=[(0,0),(1,0),(2,0),(3,0)])   ,       # flat-v
        Rock(name="+", coords=[(1,0),(0,1),(1,1),(2,1),(1,2)]),    # cross
        Rock(name="⅃", coords=[(0,0),(1,0),(2,0),(2,1),(2,2)]),    # backward-L
        Rock(name="|", coords=[(0,0),(0,1),(0,2),(0,3)])  ,        # flat-h
        Rock(name="◻", coords=[(0,0),(1,0),(0,1),(1,1)])  ,  # square
    ]
    for i in itertools.cycle(range(5)):
        yield deepcopy(ROCKS[i])

def jet_list():
    transforms = {"<":Coord(-1,0), ">":Coord(1,0)}
    jet_strings = iter(q_input[0])
    for jet in itertools.cycle(jet_strings):
        yield transforms[jet]

In [120]:
def get_rock_height(number_of_rocks):
    settled_rocks = set([Coord(x,0) for x in range(9)])
    LEFT_WALL = 0
    RIGHT_WALL = 8
    DOWN = Coord(0,-1)

    jet_gen = jet_list()
    floor = 0
    for i, rock in enumerate(rock_list(),1):
        if i == number_of_rocks +1:
            break

        #print(f"Rock {i}: {rock}")
        settled = False
        rock.transform(Coord(LEFT_WALL+3, floor+4))

        while not settled:
            # JET PUSH
            rock.transform(next(jet_gen))
            if (
                LEFT_WALL in rock.x_values or 
                RIGHT_WALL in rock.x_values or
                set(rock.coords).intersection(settled_rocks)
            ):
                rock.revert()
            
            # ROCK FALL
            rock.transform(DOWN)
            if not set(rock.coords).intersection(settled_rocks):
                continue
            
            rock.revert()
            #print(rock.coords)
            settled_rocks.update(rock.coords)
            rock_top = max(rock.y_values)
            if rock_top > floor:
                floor = rock_top
            settled = True
    return max([coord.y for coord in settled_rocks])


print("Part 1:", get_rock_height(2022))

Part 1: 3144


In [260]:
pattern = []
repeat = []
repeat_index = 0
initial_heights = []
cycle_start = False

def cycle_find(height, i):
    global pattern
    global repeat
    global repeat_index
    global cycle_start
    global initial_heights
    
    if not cycle_start and not len(initial_heights) == 156:
        initial_heights.append(height)
        return False
    if not cycle_start and len(initial_heights) == 156:
        print(f"Cycle starts at rock {i}")
        pattern.append(height)
        cycle_start = True
        return False

    if len(pattern) < 20:
        pattern.append(height)
        return False
    elif height == pattern[repeat_index]:
        repeat.append(height)
        repeat_index += 1
    else:
        pattern.extend(repeat)
        repeat = []
        repeat_index = 0
        if height == pattern[repeat_index]:
            repeat.append(height)
            repeat_index += 1
        else:
            pattern.append(height)

        return False

    if pattern == repeat:
        return True
    return False

def get_rock_height(number_of_rocks):
    settled_rocks = set([Coord(x,0) for x in range(9)])
    LEFT_WALL = 0
    RIGHT_WALL = 8
    DOWN = Coord(0,-1)

    jet_gen = jet_list()
    floor = 0
    for i, rock in enumerate(rock_list(),1):
        if i == number_of_rocks +1:
            break
            
        
        if i == 611:
            print("Remainder height:", sum(pattern))

        # if i%100  == 0:
        #     print("Rock",i)

        #print(f"Rock {i}: {rock}")
        settled = False
        rock.transform(Coord(LEFT_WALL+3, floor+4))

        while not settled:
            # JET PUSH
            rock.transform(next(jet_gen))
            if (
                LEFT_WALL in rock.x_values or 
                RIGHT_WALL in rock.x_values or
                set(rock.coords).intersection(settled_rocks)
            ):
                rock.revert()
            
            # ROCK FALL
            rock.transform(DOWN)
            if not set(rock.coords).intersection(settled_rocks):
                continue
            
            rock.revert()
            #print(rock.coords)
            settled_rocks.update(rock.coords)
            rock_top = max(rock.y_values)
            if rock_top > floor:
                if cycle_find(rock_top-floor, i):
                    return(f"CYCLE FOUND at rock {i}")
                floor = rock_top
            settled = True
    return max([coord.y for coord in settled_rocks])

print(get_rock_height(4000))
print("Initial height", sum(initial_heights))
print("Repeat height", sum(pattern))

Cycle starts at rock 218
Remainder height: 601
CYCLE FOUND at rock 3727
Initial height 322
Repeat height 2747
